<a href="https://colab.research.google.com/github/marco-siino/llm-network-testbed/blob/main/knowledge-extraction/Q1/Mistral/Mistral_LogKnowledgeExtraction_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and load the model.

In [2]:
import os
import re
import random
import time
import requests
!pip install rouge-score

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

!pip install mistralai
from mistralai import Mistral

#api_key = os.environ['7yOu0lH5XcZC1U2ZUI2uv0ghBDduhHp5']

client = Mistral(api_key='7yOu0lH5XcZC1U2ZUI2uv0ghBDduhHp5')

model = "mistral-medium"

message = [
    {
        "role": "user",
        "content": "Write a function for fibonacci"
    }
]

chat_response = client.chat.complete(
    model=model,
    messages=message
)
print(chat_response.choices[0].message.content)

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d24d1d16080a2b3c447f2ec37fae10c0a30eee413407da0cdf1283ab5b168b99
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 11.2 MB/s eta 0:00:00
Here is a simple recursive function for generating the Fibonacci sequence in Python:
```
def fibonacci(n):
  if n <= 0:
    return "Input should be a positive integer."
  elif n == 1:
    return 0
  elif n == 2:
    return 1
  else:
    return fibonacci(n-1) + fibonacci(n-2)
```
However, this function has an exponential time complexity due to the repeated computation of the same subproblems. A more efficient solution would be to use dynamic programming to store the results of previously computed Fibonacci numbers. Here is an example of an iterative function that does this:


In [3]:
!nvidia-smi

Fri Feb 28 08:58:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Download and organize the dataset.

In [4]:
url = "https://github.com/marco-siino/llm-network-testbed/raw/refs/heads/main/knowledge-extraction/Q1/prompt_Q1-2024-05-20.zip"
output_file = "prompt_Q1-2024-05-20.zip"

os.system(f"wget -O {output_file} {url}")

0

In [5]:
!unzip prompt_Q1-2024-05-20.zip

Archive:  prompt_Q1-2024-05-20.zip
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1060.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1072.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1090.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1004.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1020.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=990.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1068.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1076.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1078.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1010.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1084.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1006.txt  
  inflating: prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1070.txt  
  inflating: prom

In [6]:
folder_path="prompt_Q1-2024-05-20"
# List all files in the folder
files = os.listdir(folder_path)

# Sort the files alphabetically
files.sort()

# Function to pre-process the few shots.

In [7]:
# The following function add the Mistral Tags within the file content.
def preprocess_sample(file_contents):

                sample ="<s>[INST]Given the following LOG, how many frames are received per each dev_eui?\n\nLOG:\n"
                modified_string = file_contents.replace("QUESTION: How many packets are received for each device? ", "")
                modified_string = modified_string.replace("\n\nANSWER:", "ANSWER:[/INST]")
                modified_string = modified_string[:-1]

                sample += modified_string+"</s>"
                sample += "\n\n"

                return sample


# Create the few-shots samples.

In [8]:
def create_few_shots_set(folder_path,nr_samples):

    # Counter to keep track of the number of files read
    count = 0

    few_shots_set = ""

    # Get a list of all files in the directory
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Shuffle the list of files. Seed set - equal to 4 - for reproducibility.
    random.Random(4).shuffle(files)

    global test_set
    test_set = files

    # Iterate through all files in the folder
    for filename in files:
        print(filename)
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                few_shots_set+=preprocess_sample(file_contents)

                # Increment the counter
                count += 1

                # Check if reached the limit of n files
                if count == n:
                    break
        test_set.remove(filename)

    return few_shots_set

In [10]:
# Specify the number of files to read
n = 5  # Example: Read the first 5 files

# Call the function to read text files in the folder and concatenate their contents
few_shots_samples = create_few_shots_set(folder_path, n)

# Print or use the concatenated contents as needed
print(few_shots_samples)

prompt_n_hours=2_skip_hours=780.txt
prompt_n_hours=2_skip_hours=1068.txt
prompt_n_hours=2_skip_hours=768.txt
prompt_n_hours=2_skip_hours=306.txt
prompt_n_hours=2_skip_hours=198.txt
<s>[INST]Given the following LOG, how many frames are received per each dev_eui?

LOG:
time: 2019-06-21 12:15:29.120000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4655
time: 2019-06-21 12:19:04.772000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 24, FCnt: 202
time: 2019-06-21 12:19:32.236000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 203
time: 2019-06-21 12:29:18.989000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -107.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4906
time: 2019-06-21 12:33:13.622000, freq: 868.3, chan: 22, lsnr: -9.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4726
t

# Chat with the model to get the results.

In [11]:
# Define the directory path containing the results.
result_folder_name = 'mistral_results'
result_directory_path = os.path.join(os.getcwd(), result_folder_name)
dataset_path = 'prompt_Q1-2024-05-20'

# Create the directory if it doesn't exist
if not os.path.exists(result_directory_path):
  os.makedirs(result_directory_path)

# Iterate through all the test_files
for filename in test_set:
        print(filename)
        if(os.path.isfile(result_directory_path+"/"+filename)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Open the file and read its contents
            with open(os.path.join(dataset_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                prompt = few_shots_samples + "\n[INST]Given the following LOG, how many frames are received per each dev_eui?\n"

                # Split the string at the occurrence of "ANSWER:"
                parts = file_contents.split("ANSWER:", 1)

                parts[0] = parts[0].replace("\n\nQUESTION: How many packets are received for each device? ", "")

                # Take only until ANSWER
                prompt+= parts[0]+"ANSWER: [/INST]"

                print(prompt)

                message = [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]

                success = False
                while not success:
                    try:
                        chat_response = client.chat.complete(
                            model=model,
                            messages=message
                        )
                        success = True  # Se non viene sollevata alcuna eccezione, imposta success su True
                    except Exception as e:
                        print(f"Errore: {e}. Riprovo tra 20 secondi...")
                        time.sleep(20)  # Attendi 20 secondi prima di ritentare


                result = chat_response.choices[0].message.content
                print(result+"\n\n")

                # Define the file path
                file_path = os.path.join(result_directory_path, filename)

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

Streaming output truncated to the last 5000 lines.

<s>[INST]Given the following LOG, how many frames are received per each dev_eui?

LOG:
time: 2019-05-28 06:14:28.922000, freq: 868.1, chan: 21, lsnr: 4.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4292
time: 2019-05-28 06:18:00.251000, freq: 868.3, chan: 22, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 25, FCnt: 4305
time: 2019-05-28 06:37:42.736000, freq: 868.3, chan: 22, lsnr: -1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4299
time: 2019-05-28 06:37:48.930000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4299
time: 2019-05-28 06:44:26.964000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4250
time: 2019-05-28 06:46:14.329000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -93.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size

In [12]:
# Count and print the number of file within the codestral_results folder
file_count = sum(1 for _ in os.listdir(result_folder_name) if os.path.isfile(os.path.join(result_folder_name, _)))
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 547


In [13]:
# Count and print the number of file within the codestral_results folder
file_count = sum(1 for _ in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, _)))
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 546


# Given the same few-shot samples ask to Mistral to generate the Python code to perform the task.

In [14]:
prompt ="""
Given the following LOG string, write a Python function to calculate and output (print and store in a file) how many frames are received per each dev_eui.

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889


OUTPUT:
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 6.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 1.

"""

In [15]:
message = [
    {
        "role": "user",
        "content": prompt
    }
]

chat_response = client.chat.complete(
    model=model,
    messages=message
)

result = chat_response.choices[0].message.content
print(result+"\n\n")

Here's a Python function that calculates and outputs the number of frames received per device:
```python
def count_frames(log):
    frames_per_device = {}
    log_lines = log.split('\n')
    for line in log_lines:
        if line:
            fields = line.split(', ')
            dev_eui = fields[-2].split(': ')[1]
            if dev_eui not in frames_per_device:
                frames_per_device[dev_eui] = 0
            frames_per_device[dev_eui] += 1

    with open('frames_per_device.txt', 'w') as f:
        for dev_eui, num_frames in frames_per_device.items():
            print(f'For DEVICE dev_eui:{dev_eui} all received frames are {num_frames}.')
            f.write(f'For DEVICE dev_eui:{dev_eui} all received frames are {num_frames}.\n')

# Example usage
log = '''
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: 

In [16]:
# Let's write a function to use this code.
# To be adherent as much as possible to the original question we try to not change almost anything.
def count_frames(log):
    frames_per_device = {}
    log_lines = log.split('\n')
    for line in log_lines:
        if line:
            fields = line.split(', ')
            dev_eui = fields[-2].split(': ')[1]
            if dev_eui not in frames_per_device:
                frames_per_device[dev_eui] = 0
            frames_per_device[dev_eui] += 1

    with open('frames_per_device.txt', 'w') as f:
        for dev_eui, num_frames in frames_per_device.items():
            print(f'For DEVICE dev_eui:{dev_eui} all received frames are {num_frames}.')
            f.write(f'For DEVICE dev_eui:{dev_eui} all received frames are {num_frames}.\n')

# Example usage
log = '''
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
...
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889
'''
count_frames(log)

IndexError: list index out of range

## Let's run the code.

In [18]:
# Define the directory path containing the results.
#folder_name = 'codestral_source_code_results/'

result_folder_name = 'codestral_source_code_results'
result_directory_path = os.path.join(os.getcwd(), result_folder_name)
dataset_path = 'prompt_Q1-2024-05-20'

#directory_path = os.path.join(os.getcwd(), folder_name)
# Create the directory if it doesn't exist
if not os.path.exists(result_folder_name):
  os.makedirs(result_folder_name)

# Iterate through all the test_files
for filename in test_set:
        file_path = result_directory_path+"/"+filename
        #print(filename)
        #print(file_path)
        if(os.path.isfile(result_directory_path+"/"+filename)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Open the file and read its contents
            with open(os.path.join(dataset_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                # Remove the ANSWER in the original log file.
                file_contents = file_contents.split("ANSWER", 1)
                print(file_contents[0])

                result = count_frames(file_contents[0])

                print(str(result)+"\n\n")

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

time: 2019-06-30 08:12:42.175000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 5132
time: 2019-06-30 08:12:48.306000, freq: 868.1, chan: 21, lsnr: 2.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 5132
time: 2019-06-30 08:17:08.457000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 423
time: 2019-06-30 08:26:18.048000, freq: 868.5, chan: 23, lsnr: 11.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 5126
time: 2019-06-30 08:32:03.570000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4947
time: 2019-06-30 08:33:10.204000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -81.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 24, FCnt: 5077
time: 2019-06-30 08:33:37.600000, freq: 868.5, chan: 23, lsnr: 14.0, rssi: -82.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 12

IndexError: list index out of range

# Here the function to run the syntactic evaluation at file-level and at global-level.

In [19]:
def evaluate_results (dataset_path,result_path):
  samples_nr = 0
  global_bleu = 0
  global_rouge1_fmeasure = 0
  global_rougeL_fmeasure = 0
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      print(text2)
      print("\n")
      print(text1)

      #dataset_corpus += text2
      #results_corpus += text1

      # Tokenize the texts
      reference = text1.split()
      candidate = text2.split()

      # Compute BLEU score
      smoothing_function = SmoothingFunction().method1
      bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

      # Compute ROUGE score
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
      rouge_scores = scorer.score(text1, text2)

      # Print the results
      print(f"BLEU score: {bleu_score}")
      print(f"ROUGE-1: {rouge_scores['rouge1']}")
      print(f"ROUGE-L: {rouge_scores['rougeL']}")

      print("\n\n")


      global_bleu += bleu_score
      global_rouge1_fmeasure += rouge_scores['rouge1'].fmeasure
      global_rougeL_fmeasure += rouge_scores['rougeL'].fmeasure

      samples_nr += 1
  # Now let's print the results on the whole corpus.
  print("\n\nTHE RESULT ON THE TWO CORPORA ARE:\n")
  # Tokenize the texts
  #reference = results_corpus.split()
  #candidate = dataset_corpus.split()

  global_bleu = global_bleu/samples_nr
  global_rouge1_fmeasure = global_rouge1_fmeasure/samples_nr
  global_rougeL_fmeasure = global_rougeL_fmeasure/samples_nr

  # Print the results
  print(f"BLEU score: {global_bleu}")
  print(f"ROUGE-1: {global_rouge1_fmeasure}")
  print(f"ROUGE-L: {global_rougeL_fmeasure}")

## Let's evaluate the results of Mistral.

In [20]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "mistral_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.
For DEVICE dev_eui:126 all received frames are 3.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 6.
For DEVICE dev_eui:129 all received frames are 2.


The number of frames received for each dev_eui is as follows:

* dev_eui: 120: 2
* dev_eui: 121: 2
* dev_eui: 122: 2
* dev_eui: 123: 3
* dev_eui: 124: 5
* dev_eui: 125: 3
* dev_eui: 126: 4
* dev_eui: 127: 3
* dev_eui: 128: 6
* dev_eui: 129: 2

Note: Since there are multiple entries with the same timestamp, freq, chan, lsnr, rssi, SF, BW, codr, dev\_eui, and FCnt for dev\_eui 124, 128, and 129, I have only counted each unique entry once. If you want to count each occurrence of a frame with the same FCnt value as a separate frame, then you should update the count accordingly.
BLEU score: 0.0020237907212404213
ROUGE-1: Score(precision=0.44, recall=0.38596491228070173, fmeasure=0.411214953271028)
ROUGE-L: Score(precision=0.37, recall=0.32456

## Let's evaluate the results of the source code generated by Codestral.

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "codestral_source_code_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.

BLEU score: 0.5725114538460981
ROUGE-1: Score(precision=1.0, recall=1.0, fmeasure=1.0)
ROUGE-L: Score(precision=0.88, recall=0.88, fmeasure=0.88)



Sample:  prompt_n_hours=2_skip_hours=770.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 2.
For DEVICE dev_eui:125 all received frames are 2.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 6.
For DEVICE dev_eui:129 all received frames are 2.


For DEVICE dev_eui: 122 all received frames are 2.
For DEVICE dev_eui: 124 all received frames are 2.
For DEVICE dev_eui: 129 all received frames are 2.
For DEVICE dev_eui: 121 all received frames are 4.
For DEVICE dev_eui: 123 all received frames are 

#Let's evaluate the correctness of the responses in terms of Precision and Recall.

In [29]:
def calculate_precision_recall (dataset_path,result_path):
  total_true_positive = 0
  total_false_positive = 0
  total_false_negative = 0
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      #print(text2)
      #print("\n")
      #print(text1)

      # text1 contains results. text2 contains the correct values.
      # Create a dictionary for both text files. Key is the dev_eui and value is the number of frames.

      # For each line in text1 create a pair key value. key is the number of dev_eui, and value is the number of frames.
      results_dict = {}
      target_dict = {}

      #print(text1)

      # Split the data into lines
      lines = text1.split('\n')

      #print(lines)

      for line in lines:
        parts = line.split()
        dev_eui, frames = None, None  # Inizializza prima del blocco try
        try:
          match = re.findall(r"\d+", line)
          if len(match) >= 2:  # Assicura che ci siano almeno due numeri nella riga
            dev_eui = match[0]  # Il primo numero è il dev_eui
            frames = match[1]    # Il secondo numero è il numero di frames
        except:
          continue

        # Add to dictionary
        try:
          results_dict[dev_eui] = int(frames)
        except:
          results_dict[dev_eui] = 0

      # Split the data into lines
      lines = text2.split('\n')

      #print(lines)

      for line in lines:
        #print(line)
        # Split the line to extract dev_eui and frames
        parts = line.split()
        try:
          dev_eui = parts[2].split(':')[1]
          frames = parts[7].split('.')[0]
        except:
          continue

        # Add to dictionary
        try:
          target_dict[dev_eui] = int(frames)
        except:
          target_dict[dev_eui] = 0

      print(results_dict)
      print(target_dict)

      # Let's count the true positive.
      true_positive = 0
      for key in results_dict:
        if key in target_dict:
          if results_dict[key] == target_dict[key]:
            true_positive += 1
      print("True positive so far: ",true_positive)


      # Let's count the false positive.
      false_positive = 0
      for key in results_dict:
        if key not in target_dict:
          false_positive += 1
        else:
          if key in target_dict and results_dict[key] != target_dict[key]:
            false_positive += 1
      print("False positive so far: ",false_positive)

      # Let's count the false negative.
      false_negative = 0
      for key in target_dict:
        if key not in results_dict:
          false_negative += 1
      print("False negative so far: ",false_negative)

      total_true_positive += true_positive
      total_false_positive += false_positive
      total_false_negative += false_negative

  print("The precision is: ",total_true_positive/(total_true_positive+total_false_positive))
  print("The recall is: ",total_true_positive/(total_true_positive+total_false_negative))

##Let's evaluate Codestral

In [30]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "mistral_results/"
calculate_precision_recall(dataset_path,result_path)

Sample:  prompt_n_hours=2_skip_hours=992.txt
{None: 0, '120': 1, '121': 3, '122': 4, '123': 2, '124': 2, '125': 4, '126': 2, '127': 3, '128': 4, '129': 2}
{'120': 2, '121': 3, '122': 4, '123': 2, '124': 2, '125': 3, '126': 2, '127': 3, '128': 3, '129': 2}
True positive so far:  7
False positive so far:  4
False negative so far:  0
Sample:  prompt_n_hours=2_skip_hours=638.txt
{None: 0, '120': 1, '121': 2, '122': 1, '123': 2, '124': 2, '125': 2, '126': 3, '127': 2, '128': 5, '129': 2}
{'120': 2, '121': 2, '122': 1, '123': 2, '124': 2, '125': 2, '126': 3, '127': 2, '128': 6, '129': 2}
True positive so far:  8
False positive so far:  3
False negative so far:  0
Sample:  prompt_n_hours=2_skip_hours=1058.txt
{'120': 2, '121': 2, '122': 1, '123': 2, '124': 2, '125': 3, '126': 2, '127': 3, '128': 1, '129': 3}
{'120': 2, '121': 2, '122': 1, '123': 2, '124': 2, '125': 2, '126': 2, '127': 2, '128': 1, '129': 2}
True positive so far:  7
False positive so far:  3
False negative so far:  0
Sample:  

## Let's evaluate Codestral with CoC (Chain of Code)

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "codestral_source_code_results/"
calculate_precision_recall(dataset_path,result_path)

# Zip and download the results using the two approaches.

In [24]:
!zip -r mistral_results.zip mistral_results

updating: mistral_results/ (stored 0%)
updating: mistral_results/prompt_n_hours=2_skip_hours=318.txt (deflated 78%)
updating: mistral_results/prompt_n_hours=2_skip_hours=146.txt (deflated 58%)
updating: mistral_results/prompt_n_hours=2_skip_hours=874.txt (deflated 69%)
updating: mistral_results/prompt_n_hours=2_skip_hours=556.txt (deflated 54%)
updating: mistral_results/prompt_n_hours=2_skip_hours=30.txt (deflated 87%)
updating: mistral_results/prompt_n_hours=2_skip_hours=622.txt (deflated 80%)
updating: mistral_results/prompt_n_hours=2_skip_hours=682.txt (deflated 80%)
updating: mistral_results/prompt_n_hours=2_skip_hours=868.txt (deflated 82%)
updating: mistral_results/prompt_n_hours=2_skip_hours=886.txt (deflated 81%)
updating: mistral_results/prompt_n_hours=2_skip_hours=614.txt (deflated 59%)
updating: mistral_results/prompt_n_hours=2_skip_hours=228.txt (deflated 69%)
updating: mistral_results/prompt_n_hours=2_skip_hours=490.txt (deflated 80%)
updating: mistral_results/prompt_n_hou

In [ ]:
!zip -r codestral_results.zip codestral_results
!zip -r codestral_source_code_results.zip codestral_source_code_results

updating: codestral_results/ (stored 0%)
updating: codestral_results/prompt_n_hours=2_skip_hours=74.txt (deflated 79%)
updating: codestral_results/prompt_n_hours=2_skip_hours=100.txt (deflated 77%)
updating: codestral_results/prompt_n_hours=2_skip_hours=36.txt (deflated 78%)
updating: codestral_results/prompt_n_hours=2_skip_hours=760.txt (deflated 82%)
updating: codestral_results/prompt_n_hours=2_skip_hours=578.txt (deflated 79%)
updating: codestral_results/prompt_n_hours=2_skip_hours=412.txt (deflated 75%)
updating: codestral_results/prompt_n_hours=2_skip_hours=456.txt (deflated 79%)
updating: codestral_results/prompt_n_hours=2_skip_hours=972.txt (deflated 81%)
updating: codestral_results/prompt_n_hours=2_skip_hours=438.txt (deflated 81%)
updating: codestral_results/prompt_n_hours=2_skip_hours=586.txt (deflated 80%)
updating: codestral_results/prompt_n_hours=2_skip_hours=4.txt (deflated 79%)
updating: codestral_results/prompt_n_hours=2_skip_hours=294.txt (deflated 77%)
updating: codes